## Need to be running a python  version that works with outlier_utils

[Resource](https://www.alfredo.motta.name/create-isolated-jupyter-ipython-kernels-with-pyenv-and-virtualenv/) for having multiple versions of python + Jupyter. The colab version is 3.6.9 but not an option in pyenv so went with 3.6.8

In [6]:
from platform import python_version

print(python_version())

3.6.8


## Update path_j based on user & install packages

In [2]:
#google drive authorize AND googlesheets4 authorize-- must create a project under google API and then download credentials
#see authentication section in the following article
#https://medium.com/@bretcameron/how-to-use-the-google-drive-api-with-javascript-57a6cc9e5262
#Note: need to open a service account and extract .json credentials in the API console, approve Google Drive API, then share the service account email address with the sheet of interest
path_j='/Users/owner/Desktop/Berkeley_Work/Projects/2017_San_Diego/June 2018_Data_Analysis/access-via-jupiter-notebook-d18b3ab0993b.json' #lauren's


#need  to do the following  in the activated virtual environment
# !pip install gspread
# !pip install oauth2client
# !pip install outliers
# !pip install plotnine
# !pip install scikit-misc
# !pip install outlier_utils
# !pip install pandas
# !pip  install numpy
# !pip install sklearn

In [3]:
import sys
import os
from os import path
from datetime import date
import math
import warnings
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from plotnine import *
from scipy import stats
from scipy.stats import linregress
import gspread
from mizani.formatters import scientific_format
%matplotlib inline


#Set up Google drive access and sheets manipulation

import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name(path_j, scope)


#Authorize credentials and open master sampling table using its URL
gc = gspread.authorize(credentials)
# routine= gc.open_by_url('https://docs.google.com/spreadsheets/d/163xXZnJjh0LdryxrDhuYxgzHB_IY7Jaew3DpYzoNs9Q/edit#gid=1255279898')
# qpcr=gc.open_by_url('https://docs.google.com/spreadsheets/d/1TzZcB2p55BuG5FbXAA44EFoe8BdRapGyDaVR4hx9NXU/edit#gid=1689469809')
import skmisc
import outliers

#sheets
rna_tab = 'sample_inventory'
ww_tab='site_lookup'
facility_lookup='site_lookup'
qpcr_results_tab = 'QuantStudio_raw_data'
qpcr_plates_tab = 'Plate_info'

#urls
samples_url = 'https://docs.google.com/spreadsheets/d/163xXZnJjh0LdryxrDhuYxgzHB_IY7Jaew3DpYzoNs9Q'
qpcr_url = 'https://docs.google.com/spreadsheets/d/1TzZcB2p55BuG5FbXAA44EFoe8BdRapGyDaVR4hx9NXU'


# read in custom modules
from read_gsheets import * 
from reprocess_qpcr import *
from calculations import *
from qa_qc import *

In [4]:
# load data for plotting trends over time
# read in data
sample_data = read_sample_data(gc, samples_url, rna_tab, facility_lookup)
qpcr_raw = read_qpcr_data(gc, qpcr_url, qpcr_results_tab, qpcr_plates_tab)
qpcr_processed, std_curve_df, raw_outliers_flagged_df = process_qpcr_raw(qpcr_raw, 'grubbs_only')

# merge with sample data
qpcr_averaged = qpcr_processed.merge(sample_data, how='left', left_on='Sample', right_on='sample_id')
qpcr_averaged = qpcr_averaged[qpcr_averaged.Sample != 'NTC']
qpcr_averaged_xeno = qpcr_averaged
qpcr_averaged = qpcr_averaged[qpcr_averaged.Target != 'Xeno'] #only need in raw_outliers_flagged_df

# calculations
qpcr_averaged['gc_per_L'] = calculate_gc_per_l(qpcr_averaged) # get gc/L
qpcr_averaged = normalize_to_pmmov(qpcr_averaged)
qpcr_averaged = get_GFP_recovery(qpcr_averaged)
qpcr_averaged=qpcr_averaged[~(qpcr_averaged['batch'].str.contains("R", na=True))].copy() #only variability batches and archive stuff

#make column for PBS controls assessment
PBS=qpcr_averaged[qpcr_averaged.interceptor=='PBS'][['plate_id','batch','Target','Cq_init_min','Cq_of_lowest_std_quantity','ntc_result']].copy()
PBS.Cq_init_min= pd.to_numeric(PBS.Cq_init_min)
PBS['PBS_result']=np.nan
PBS.loc[np.isnan(PBS.Cq_init_min),"PBS_result"]="negative"
PBS.loc[~np.isnan(PBS.Cq_init_min),"PBS_result"]=PBS.loc[~np.isnan(PBS.Cq_init_min),"Cq_init_min"]

mer=PBS[['plate_id','batch',"Target","PBS_result"]].copy()

qpcr_averaged=qpcr_averaged[qpcr_averaged.interceptor!='PBS'].copy()
qpcr_averaged_merged=qpcr_averaged.merge(mer, how='left')


sample_data["processed"]=0
sample_data.loc[sample_data.date_extract != "","processed"]=1
sample_data["N1_qpcr"]=0
uni=qpcr_averaged[(qpcr_averaged.Target == 'N1')].Sample.unique()
sample_data.loc[sample_data.sample_id.isin(uni) ,"N1_qpcr"]=1
sample_data["paper"]=0
num=['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11','B12','B13','B14','B15']
sample_data.loc[sample_data.batch.isin(num),"paper"]=1
sample_data["sub_id"]=sample_data['sample_id'].str.slice(0, -2)

sample_data_80=sample_data[sample_data.stored_minus_80=='1'].copy()
sample_data=sample_data[sample_data.processed_fresh != '']
sample_data.processed_fresh=sample_data.processed_fresh.astype('int')
sample_data_fresh=sample_data[sample_data.processed_fresh==1].copy()
sample_data_u=sample_data_80[['facility',"interceptor",'date_sampling','date_extract','processed',"N1_qpcr","paper"]].drop_duplicates()
sample_data_fresh_u=sample_data_fresh[['facility',"interceptor",'date_sampling','date_extract','processed',"N1_qpcr","paper"]].drop_duplicates()


sample_data=sample_data[(sample_data.processed_fresh != 1)&(~np.isnan(sample_data.processed_fresh))].copy()


qpcr_averaged_data = qpcr_averaged[qpcr_averaged.inhibition_testing == "N"] # Remove inhibition testing from qPCR averaged
qpcr_unk_long_data = qpcr_averaged_data[qpcr_averaged_data.Task=='Unknown'].copy().reset_index() # filter out standards and copy


/Users/owner/Desktop/Berkeley_Work/data_analysis/calculations.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/owner/Desktop/Berkeley_Work/data_analysis/calculations.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/owner/.pyenv/versions/py3.6.8_outlierutils/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [5]:
#inhibition testing
xeno_inhib_full, xeno_control=xeno_inhibition_test(raw_outliers_flagged_df)

xeno_inhib=xeno_inhib_full.merge(sample_data, left_on='Sample', right_on='sample_id',  how='left').copy()
inhibited=xeno_inhib[xeno_inhib.dCt>1].Sample.unique()
not_inhibited=xeno_inhib[xeno_inhib.dCt<=1].Sample.unique()

qpcr_averaged_merged["is_inhibited"]='unknown'
qpcr_averaged_merged.loc[qpcr_averaged_merged.Sample.isin(inhibited),"is_inhibited"]="Yes"
qpcr_averaged_merged.loc[qpcr_averaged_merged.Sample.isin(not_inhibited),"is_inhibited"]="No"

In [8]:
qpcr_averaged_merged.head()

,plate_id,Sample,Sample_plate,Target,Task,inhibition_testing,template_volume,Q_init_mean,Q_init_std,Q_init_CoV,...,interceptor_general_flow_MGD,gc_per_ul_input,gc_per_L,pmmov_mean,mean_normalized_to_pmmov,log10mean_normalized_to_log10pmmov,log10_mean_normalized_to_pmmov,perc_GFP_recovered,PBS_result,is_inhibited
0,41,D_CH_081320_1,D_CH_081320_1+41,N1,Unknown,N,5.0,243.829916,1.040093,0.031818,...,,46.395615,231978.076337,340790.729318,0.000681,0.427556,-3.167041,NaN,NaN,unknown
1,41,D_CH_081320_2,D_CH_081320_2+41,N1,Unknown,N,5.0,232.638130,1.067920,0.054632,...,,42.327009,211635.042963,408204.186208,0.000518,0.414478,-3.285290,NaN,NaN,unknown
2,41,D_CH_081320_3,D_CH_081320_3+41,N1,Unknown,N,5.0,287.840558,1.070694,0.054703,...,,53.954251,269771.257496,369573.431943,0.000730,0.436625,-3.136705,NaN,NaN,unknown
3,41,D_CH_081320_4,D_CH_081320_4+41,N1,Unknown,N,5.0,219.437214,1.099899,0.078809,...,,38.805509,194027.546072,476033.892406,0.000408,0.402960,-3.389774,NaN,NaN,unknown
4,41,D_CH_081320_5,D_CH_081320_5+41,N1,Unknown,N,5.0,195.834056,1.022019,0.017883,...,,37.471137,187355.686688,342977.970025,0.000546,0.410580,-3.262599,NaN,NaN,unknown
